In [1]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
import os

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib


In [3]:
def load_images(folder_path, label):
    images = []
    labels = []
    
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))  # Resize image to a fixed size
        images.append(img.flatten())  # Flatten the image matrix
        labels.append(label)
    
    return images, labels

# Load images and labels
cat_images, cat_labels = load_images('D:\Internships\Prodigy_InfoTech\cats', 'cat')
dog_images, dog_labels = load_images('D:\Internships\Prodigy_InfoTech\dogs', 'dog')


In [4]:
X = np.concatenate((cat_images, dog_images), axis=0)
y = np.concatenate((cat_labels, dog_labels), axis=0)


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [7]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train_encoded)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)



In [8]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("D:\Internships\Prodigy_InfoTech\keras_Model.h5", compile=False)

# Load the labels
class_names = open("D:\Internships\Prodigy_InfoTech\labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("D:\Internships\Prodigy_InfoTech\cats\cat.275.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


1/1 [==============================] - 1s 1s/step
Class: Cats
Confidence Score: 1.0


In [11]:
import gradio as gr

# Load the model and labels
model = load_model("D:\Internships\Prodigy_InfoTech\keras_Model.h5", compile=False)
class_names = open("D:\Internships\Prodigy_InfoTech\labels.txt", "r").readlines()

# Define the prediction function
def classify_image(input_image):
    # Preprocess the input image
    image_array = input_image.astype(np.uint8)
    image = Image.fromarray(image_array).convert("RGB")
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array

    # Make prediction
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index][2:]
    confidence_score = prediction[0][index]

    return f"Class: {class_name} | Confidence Score: {confidence_score:.4f}"

# Create the Gradio interface
iface = gr.Interface(fn=classify_image, inputs="image", outputs="text", live=True)

# Launch the interface
iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 63ms/step
